In [6]:
import tkinter as tk
from tkinter import ttk
from tkinter.messagebox import showinfo

# root window
root = tk.Tk()
root.eval('tk::PlaceWindow . center')
root.geometry("300x250")
root.resizable(False, False)
root.title('Site Details')

# store email address and password
Username = tk.StringVar()
password = tk.StringVar()
IP_Address = tk.StringVar()
Site_code = tk.StringVar()

# Sign in frame
signin = ttk.Frame(root)
signin.pack(padx=10, pady=10, fill='x', expand=True)


# Username
Username_label = ttk.Label(signin, text="Username:")
Username_label.pack(fill='x', expand=True)

Username_entry = ttk.Entry(signin, textvariable=Username)
Username_entry.pack(fill='x', expand=True)
Username_entry.focus()

# password
password_label = ttk.Label(signin, text="Password:" )
password_label.pack(fill='x', expand=True)

password_entry = ttk.Entry(signin, textvariable=password, show="*")
password_entry.pack(fill='x', expand=True)

# IP Address
IP_Address_label = ttk.Label(signin, text="IP Address:")
IP_Address_label.pack(fill='x', expand=True)

IP_Address_entry = ttk.Entry(signin, textvariable=IP_Address)
IP_Address_entry.pack(fill='x', expand=True)

# Site Code
Site_code_label = ttk.Label(signin, text="Site code:")
Site_code_label.pack(fill='x', expand=True)

Site_code_entry = ttk.Entry(signin, textvariable=Site_code)
Site_code_entry.pack(fill='x', expand=True)
Site_code_entry.focus()

# login button
login_button = ttk.Button(signin, text="Login", command=root.destroy)
login_button.pack(fill='x', expand=True, pady=10)

root.attributes('-topmost', True)
root.mainloop()

In [12]:
email_var = email.get()
pass_var = password.get()
IP_Address = IP_Address.get()
Site_code = Site_code.get()

In [14]:
Site_code

''